In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
print(os.listdir(('/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/train/images/')))

['class 0', 'class 1']


In [ ]:
!pip install torch torchvision opencv-python
!pip install scikit-plot
!pip install django-model-utils

In [3]:
import os
import torch
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Definir a arquitetura do encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = torchvision.models.resnet18(pretrained=True)
        self.encoder.fc = nn.Identity()  # Remover a última camada totalmente conectada

    def forward(self, x):
        x = self.encoder(x)
        return x

# Definir a arquitetura do decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Linear(512, num_classes)  # Substituir 512 pelo número de classes do seu problema

    def forward(self, x):
        x = self.decoder(x)
        return x

# Função para carregar as imagens e anotações
def load_data(root_dir):
    image_dir_train = os.path.join(root_dir, 'train/images')
    annotation_dir_train = os.path.join(root_dir, 'train/anotacoes')
    image_dir_test = os.path.join(root_dir, 'valid/images')
    annotation_dir_test = os.path.join(root_dir, 'valid/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])

    train_data = torchvision.datasets.ImageFolder(
        root=image_dir_train,
        transform=transform
    )

    test_data = torchvision.datasets.ImageFolder(
        root=image_dir_test,
        transform=transform
    )

    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=16,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=16,
        shuffle=False
    )

    return train_loader, test_loader

# Definir os parâmetros
root_dir = '/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/'
num_classes = 2  # Substituir pelo número de classes do seu problema
encoder_lr = 0.01
decoder_lr = 0.01
num_epochs = 3

# Carregar os dados
train_loader, test_loader = load_data(root_dir)

# Inicializar o encoder e o decoder
encoder = Encoder()
decoder = Decoder()

# Definir a função de perda e os otimizadores
criterion = nn.CrossEntropyLoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=encoder_lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=decoder_lr)

# Treinamento
for epoch in range(num_epochs):
    encoder.train()
    decoder.train()

    for images, labels in train_loader:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Passar as imagens pelo encoder
        features = encoder(images)

        # Passar as features pelo decoder
        outputs = decoder(features)

        # Calcular a perda
        loss = criterion(outputs, labels)

        # Retropropagação e otimização
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    # Avaliação
encoder.eval()
decoder.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        # Passar as imagens pelo encoder
        features = encoder(images)

        # Passar as features pelo decoder
        outputs = decoder(features)

        # Calcular as predições
        _, predicted = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calcular métricas
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels, target_names=[f'Class {i}' for i in range(num_classes)])

# Imprimir métricas
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# Visualizar matriz de confusão
df_cm = pd.DataFrame(conf_matrix, index=[i for i in range(num_classes)], columns=[i for i in range(num_classes)])
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True, fmt="d")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Salvar os modelos treinados
torch.save(encoder.state_dict(), 'encoder.pth')
torch.save(decoder.state_dict(), 'decoder.pth')


FileNotFoundError: ignored

In [7]:
import os
import torch
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import os
import torch
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define a arquitetura da U-Net
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # Encoder
        self.encoder = torchvision.models.resnet18(pretrained=True)
        self.encoder.fc = nn.Identity()  # Remove a camada fully connected

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 1, kernel_size=1)  # Saída binária para segmentação
        )

    def forward(self, x):
        # Encoder
        x = self.encoder(x)

        # Decoder
        x = self.decoder(x)
        return x

# Função para carregar as imagens e anotações
def load_data(root_dir):
    image_dir_train = os.path.join(root_dir, 'train/images')
    annotation_dir_train = os.path.join(root_dir, 'train/anotacoes')


    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])

    train_data = torchvision.datasets.ImageFolder(
        root=image_dir_train,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=16,
        shuffle=True
    )
    return train_loader

def test_load_data(root_dir):
    image_dir_test = os.path.join(root_dir, 'test/images')
    annotation_dir_test = os.path.join(root_dir, 'test/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])
    test_data = torchvision.datasets.ImageFolder(
        root=image_dir_test,
        transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=16,
        shuffle=False
    )

    return test_loader

def evaluate_model(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

def validation_load_data(root_dir):  # Corrigir o nome do parâmetro para root_dir
    image_dir_val = os.path.join(root_dir, 'valid/images')
    annotation_dir_val = os.path.join(root_dir, 'valid/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    val_data = torchvision.datasets.ImageFolder(
        root=image_dir_val,
        transform=transform
    )
    val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=16,
        shuffle=False
    )

    return val_loader


# Dentro da função train_model, substitua a criação do modelo ResNet pela criação do modelo Encoder-Decoder
def train_model(train_loader, val_loader, num_epochs, MODELO):
    model = UNet()  # Use a U-Net em vez de ResNet
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()  # Use uma função de perda apropriada para segmentação binária
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


    best_accuracy = 0.0  # Acompanhe a melhor precisão de validação
    train_accuracies = []  # Lista para armazenar as precisões de treinamento
    train_losses = []  # Lista para armazenar as perdas de treinamento

    best_val_accuracy = 0.0  # Acompanhe a melhor precisão de validação
    val_accuracies = []  # Lista para armazenar as precisões de validação

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        train_accuracy = evaluate_model(model, train_loader)
        train_accuracies.append(train_accuracy)

        # Validação após cada época
        model.eval()
        val_accuracy = evaluate_model(model, val_loader)
        val_accuracies.append(val_accuracy)

        print(f'Época [{epoch+1}/{num_epochs}], Precisão de Treinamento: {train_accuracy:.2%}, Precisão de Validação: {val_accuracy:.2%}, Perda: {epoch_loss:.4f}')

        # Salvar o melhor modelo com base na maior precisão de validação
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_state = model.state_dict()

    # Salvar o checkpoint do melhor modelo
    torch.save(best_model_state, MODELO)

    return model


def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define o dispositivo de execução
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            # Obter as probabilidades de saída (scores)
            scores = F.softmax(outputs, dim=1)
            y_scores.extend(scores[:, 1].cpu().numpy())

    # Calcula a matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Acurácia:", accuracy)
    print("Precisão:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    print("True Negative (TN):", tn)
    print("False Positive (FP):", fp)
    print("False Negative (FN):", fn)
    print("True Positive (TP):", tp)
    class_names = ['Classe 0 (Sem Glaucoma)', 'Classe 1 (Com Glaucoma)']

    # Plot da matriz de confusão
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Matriz de Confusão')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Calcular as estatísticas ROC
    fpr, tpr, thresholds_roc = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)

    # Plot do gráfico ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label='Curva ROC (AUC = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.title('Gráfico ROC')
    plt.legend(loc='lower right')
    plt.show()

    # Calcular as estatísticas RC
    precision, recall, thresholds_rc = precision_recall_curve(y_true, y_scores)

    # Plot do gráfico RC
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label='Curva RC')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Gráfico RC')
    plt.legend(loc='lower left')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))


Valid_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
Teste_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
num_epochs = 3

Val_loader = validation_load_data(Valid_root)
Test_loader = test_load_data(Teste_root)

# Carregar os dados para G1020
TrainG1020_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
Train_loader_G1020 = load_data(TrainG1020_root)
G1020M = 'G1020best_model.pth'
modelo_G1020 = train_model(Train_loader_G1020, Val_loader, num_epochs,G1020M)
melhor_estado_modelo_G1020 = torch.load(G1020M)
modelo_G1020.load_state_dict(melhor_estado_modelo_G1020)
test_model(modelo_G1020, Test_loader)

# Carregar os dados para ORIGA
TrainORIGA_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
train_loader_ORIGA = load_data(TrainORIGA_root)
ORIGM = 'ORIGAbest_model.pth'
modelo_ORIGA = train_model(train_loader_ORIGA, Val_loader, num_epochs,ORIGM)
melhor_estado_modelo_ORIGA = torch.load(ORIGM)
modelo_ORIGA.load_state_dict(melhor_estado_modelo_ORIGA)
test_model(modelo_ORIGA, Test_loader)

# Carregar os dados para REFUGE
TrainREFUGE_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/REFUGE/"
train_loader_REFUGE = load_data(TrainREFUGE_root)
REFUM = 'REFUGEbest_model.pth'
modelo_REFUGE = train_model(train_loader_REFUGE, Val_loader, num_epochs,REFUM)
melhor_estado_modelo_REFUGE = torch.load('REFUGEbest_model.pth')
modelo_REFUGE.load_state_dict(melhor_estado_modelo_REFUGE)
test_model(modelo_REFUGE, Test_loader)

RuntimeError: ignored